In [3]:
import sys,importlib
sys.path.append('../')
import simulation
from simulation import dropna
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go
import pandas

In [14]:
master = pandas.read_csv('./fort_longer.123',sep='\t')

In [15]:
master['dX'] = master['x'].diff()
master['dY'] = master['y'].diff()
master['dZ'] = master['z'].diff()

In [16]:
trace = go.Scatter3d(x=master['dX'],y=master['dY'],z=master['dZ'],mode='lines')
py.iplot([trace])

#### synchrony

$\phi = x_M - x_S - D \\$
$D = a_0 + a_1 sin(\omega t)$

In [3]:
a0,a1,w = 3.5,2.5,1*pi
epsilon = 1.5
delTime = .1

def zetaInvar(stateA,stateB,t):
    return -asarray([w*a1*cos(w*t)]*len(stateA))
def chi(stateA,stateB,t):
    return stateA-stateB-a0-a1*sin(w*t)
def zetaNull(stateA,stateB,t):
    return asarray([0,0,0],dtype=float)

NameError: name 'pi' is not defined

## slave evolution

In [18]:
def slaveEvolution(master,zeta):
    slave = zeros((len(master),3))
    stateB = master[['x','y','z']].iloc[0].to_numpy()
    #slave[0] = stateB
    for index,master_state in master.iloc[1:].iterrows():
        time = master_state['t']
        stateB = slave[index-1]
        stateA = master_state[['x','y','z']].to_numpy()

        dA = master_state[['dX','dY','dZ']].to_numpy()
        coupling = zeta(stateA,stateB,time) 
        coupling += chi(stateA,stateB,time)*epsilon
        stateB += delTime*(dA+coupling)
        slave[index] = stateB
    slave = pandas.DataFrame(slave,columns=['x','y','z'])
    return slave

In [1]:
def slaveEvolution(master,zeta):
    slave = zeros((len(master),3))
    stateB = master[['x','y','z']].iloc[0].to_numpy()
    for index,master_state in master.iloc[1:].iterrows():
        time = master_state['t']
        stateB = slave[index-1]
        stateA = master_state[['x','y','z']].to_numpy()

        dA = master_state[['dX','dY','dZ']].to_numpy()
        
        k1 = dA + chi(stateA,stateB,time) + zeta(stateA,stateB,time)
        #l1 = dA
        k2 = dA + chi(stateA + delTime*k1/2,stateB + delTime*dA/2,time + delTime/2) + zeta(stateA + delTime*k1/2,stateB + delTime*dA/2,time + delTime/2)
        #l2 = interpolated value of velocity of master at t+h/2 
        k3 = dA + chi(stateA + delTime*k2/2,stateB + delTime*dA/2,time + delTime/2) + zeta(stateA + delTime*k2/2,stateB + delTime*dA/2,time + delTime/2)
        k4 = dA + chi(stateA + delTime*k3,stateB + delTime*dA,time + delTime)

        coupling = 1/6*(k1+2*k2+2*k3+k4)
        stateB += delTime*coupling
        slave[index] = stateB
    slave = pandas.DataFrame(slave,columns=['x','y','z'])
    return slave

In [2]:
slave_zeta = slaveEvolution(master,zetaInvar)
slave_null = slaveEvolution(master,zetaNull)
D = a0-a1*sin(w*master['t'])
slave_truth = master[['x','y','z']] - expand_dims(D,axis=1)

NameError: name 'master' is not defined

In [21]:
master_trace = go.Scatter3d(name='master',x=master['x'],y=master['y'],z=master['z'],mode='lines')
slave_zeta_trace = go.Scatter3d(name='slave_zeta',x=slave_zeta['x'],y=slave_zeta['y'],z=slave_zeta['z'],mode='lines')
slave_null_trace = go.Scatter3d(name='slave_null',x=slave_null['x'],y=slave_null['y'],z=slave_null['z'],mode='lines')
slave_truth_trace = go.Scatter3d(name='true',x=slave_truth['x'],y=slave_truth['y'],z=slave_truth['z'],mode='lines')
py.iplot([master_trace,slave_zeta_trace,slave_null_trace,slave_truth_trace])

## synchrony error

In [22]:
dimension = 'y'
py.iplot([go.Scatter(name='truth_'+dimension,x=master['t'],y=slave_truth[dimension]),
         go.Scatter(name='zeta_'+dimension,x=master['t'],y=slave_zeta[dimension].shift(0)),
         go.Scatter(name='null_'+dimension,x=master['t'],y=slave_null[dimension].shift(0))])

In [70]:
error_zeta = simulation.syncError(slave_truth[['x','y','z']].to_numpy(),slave_zeta[['x','y','z']].shift(-5).to_numpy())
error_null = simulation.syncError(slave_truth[['x','y','z']].to_numpy(),slave_null[['x','y','z']].shift(-5).to_numpy())

In [71]:
py.iplot([go.Scatter(name='x',x=master['t'],y=error_zeta[:,0],mode='lines'),
         go.Scatter(name='y',x=master['t'],y=error_zeta[:,1],mode='lines'),
         go.Scatter(name='z',x=master['t'],y=error_zeta[:,2],mode='lines')])

In [72]:
py.iplot([go.Scatter(name='x',x=master['t'],y=error_null[:,0],mode='lines'),
         go.Scatter(name='y',x=master['t'],y=error_null[:,1],mode='lines'),
         go.Scatter(name='z',x=master['t'],y=error_null[:,2],mode='lines')])

In [73]:
print('error zeta:',dropna(error_zeta.sum(axis=1)).mean())
print('error null:',dropna(error_null.sum(axis=1)).mean())

error zeta: 1.786440183663626
error null: 71.55842272982665


In [69]:
print(simulation.lagMinimaSyncError(slave_truth,slave_zeta,10))
print(simulation.lagMinimaSyncError(slave_truth,slave_null,10))

(1.7801845151865627, -7)
(10.829974586075341, -7)


In [38]:
error_zeta_trace = go.Scatter(name='zeta',x=master['t'],y=error_zeta.sum(axis=1))
error_null_trace = go.Scatter(name='null',x=master['t'],y=error_null.sum(axis=1))
py.iplot([error_zeta_trace,error_null_trace])

--------------------